# 第一部分 目标检测基础

## 什么是目标检测

目标检测是计算机视觉中的一个重要任务，近年来传统目标检测方法已经难以满足人们对目标检测效果的要求，随着深度学习在计算机视觉任务上取得的巨大进展，目前基于深度学习的目标检测算法已经成为主流。

相比较于基于深度学习的图像分类任务，目标检测任务更具难度。

具体区别如图3-1所示。

图像分类：只需要判断输入的图像中是否包含感兴趣物体。

目标检测：需要在识别出图片中目标类别的基础上，还要精确定位到目标的具体位置，并用外接矩形框标出。

![avatar](./pic/3-1.png)

## 目标检测的思路
自2012年Alex Krizhevsky凭借Alex在ImageNet图像分类挑战赛中拿下冠军之后，深度学习在图像识别尤其是图像分类领域开始大放异彩，大众的视野也重新回到深度神经网络中。紧接着，不断有更深更复杂的网络出现，一再刷新ImageNet图像分类比赛的记录。

大家发现，通过合理的构造，神经网络可以用来预测各种各样的实际问题。于是人们开始了基于CNN的目标检测研究, 但是随着进一步的探索大家发现，似乎CNN并不善于直接预测坐标信息。并且一幅图像中可能出现的物体个数也是不定的，模型如何构建也比较棘手。

因此，人们就想，如果知道了图中某个位置存在物体，再将对应的局部区域送入到分类网络中去进行判别，那我不就可以知道图像中每个物体的位置和类别了吗？

但是，怎么样才能知道每个物体的位置呢？显然我们是没办法知道的，但是我们可以去猜啊！所谓猜，其实就是通过滑窗的方式，罗列图中各种可能的区域，一个个去试，分别送入到分类网络进行分类得到其类别，同时我们会对当前的边界框进行微调，这样对于图像中每个区域都能得到（class,x1,y1,x2,y2）五个属性，汇总后最终就得到了图中物体的类别和坐标信息。

总结一下我们的这种方案思路：先确立众多候选框，再对候选框进行分类和微调。

观察下图，更形象的理解下这种思想：

![avatar](./pic/3-2.png)

展示了一个通过遍历各个区域，然后逐个分类去完成目标检测任务的过程示例。在待识别图上预设一个框，然后逐像素遍历，就能得到大量候选框（这里仅为示意图，图上只展示了3个框用于说明问题，具体数量由图像大小和预设框大小决定），每个框送入到分类网络分类都有一个得分(代表当前框中有一个船的置信度)，那么得分最高的就代表识别的最准确的框，其位置就是最终要检测的目标的位置。

## 目标框定义方式

任何图像任务的训练数据都要包括两项，图片和真实标签信息，通常叫做GT。

图像分类中，标签信息是类别。目标检测的标签信息除了类别label以外，需要同时包含目标的位置信息，也就是目标的外接矩形框bounding box。

用来表达bbox的格式通常有两种，(x1, y1, x2, y2) 和 (c_x, c_y, w, h) 

![avatar](./pic/3-3.png)

之所以使用两种不同的目标框信息表达格式，是因为两种格式会分别在后续不同场景下更加便于计算。

两种格式互相转换的实现在utils.py中，代码也非常简单

In [ ]:
def xy_to_cxcy(xy):
    """
    Convert bounding boxes from boundary coordinates (x_min, y_min, x_max, y_max) to center-size coordinates (c_x, c_y, w, h).

    :param xy: bounding boxes in boundary coordinates, a tensor of size (n_boxes, 4)
    :return: bounding boxes in center-size coordinates, a tensor of size (n_boxes, 4)
    """
    return torch.cat([(xy[:, 2:] + xy[:, :2]) / 2,  # c_x, c_y
                      xy[:, 2:] - xy[:, :2]], 1)  # w, h


def cxcy_to_xy(cxcy):
    """
    Convert bounding boxes from center-size coordinates (c_x, c_y, w, h) to boundary coordinates (x_min, y_min, x_max, y_max).

    :param cxcy: bounding boxes in center-size coordinates, a tensor of size (n_boxes, 4)
    :return: bounding boxes in boundary coordinates, a tensor of size (n_boxes, 4)
    """
    return torch.cat([cxcy[:, :2] - (cxcy[:, 2:] / 2),  # x_min, y_min
                      cxcy[:, :2] + (cxcy[:, 2:] / 2)], 1)  # x_max, y_max


## 交并比（IoU）
在目标检测任务中，关于IOU的计算贯穿整个模型的训练测试和评价过程，是非常非常重要的一个概念，其目的是用来衡量两个目标框的重叠程度。

IoU的全称是交并比（Intersection over Union），表示两个目标框的交集占其并集的比例。下图为IOU计算示意图:

![avarter](./pic/3-4.png)

图中可以看到，分子中黄色区域为红bbox和绿bbox的交集，分母中黄+红+绿区域为红bbox和绿bbox的并集，两者之比即为iou。

那么具体怎么去计算呢？这里给出计算流程的简述:

1. 首先获取两个框的坐标，红框坐标: 左上(red_x1, red_y1), 右下(red_x2, red_y2)，绿框坐标: 左上(green_x1, green_y1)，右下(green_x2, green_y2)
2. 计算两个框左上点的坐标最大值:(max(red_x1, green_x1), max(red_y1, green_y1)), 和右下点坐标最小值:(min(red_x2, green_x2), min(red_y2, green_y2))
3. 利用2算出的信息计算黄框面积：yellow_area
4. 计算红绿框的面积：red_area 和 green_area
5. iou = yellow_area / (red_area + green_area - yellow_area)

代码如下：

In [ ]:
def find_intersection(set_1, set_2):
    """ 
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)                                                                                                           
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(set_1[:, :2].unsqueeze(1), set_2[:, :2].unsqueeze(0))  # (n1, n2, 2)
    upper_bounds = torch.min(set_1[:, 2:].unsqueeze(1), set_2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  # (n1, n2, 2)
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)


def find_jaccard_overlap(set_1, set_2):
    """ 
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    # PyTorch auto-broadcasts singleton dimensions
    union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - intersection  # (n1, n2)

    return intersection / union  # (n1, n2)


# 第二部分 目标检测数据集VOC

## VOC数据集简介

VOC数据集是目标检测领域最常用的标准数据集之一，几乎所有检测方向的论文，如faster_rcnn、yolo、SSD等都会给出其在VOC数据集上训练并评测的效果。因此我们我们的教程也基于VOC来开展实验，具体地，我们使用VOC2007和VOC2012这两个最流行的版本作为训练和测试的数据。

<b>数据集类别</b>

VOC数据集在类别上可以分为4大类，20小类，其类别信息如图所示。

![avarter](./pic/3-5.png)

<b>数据集量级</b>

VOC数量集图像和目标数量的基本信息如下图所示:

![avarter](./pic/3-6.png)

Images表示图片数量，Objects表示目标数量

## 数据下载地址
https://pan.baidu.com/s/1SM2M6S366igDriiCWGyhDg 提取码：7aek

## 安装方法
下载后放到dataset目录下解压即可

<b>数据集说明</b>

将下载得到的压缩包解压，可以得到如图3-9所示的一系列文件夹，由于VOC数据集不仅被拿来做目标检测，也可以拿来做分割等任务，因此除了目标检测所需的文件之外，还包含分割任务所需的文件，比如SegmentationClass,SegmentationObject，这里，我们主要对目标检测任务涉及到的文件进行介绍。

![avarter](./pic/3-9.png)

<b>1.JPEGImages</b>

这个文件夹中存放所有的图片，包括训练验证测试用到的所有图片。

<b>2.ImageSets</b>

这个文件夹中包含三个子文件夹，Layout、Main、Segmentation

Layout文件夹中存放的是train，valid，test和train+valid数据集的文件名

Segmentation文件夹中存放的是分割所用train，valid，test和train+valid数据集的文件名

Main文件夹中存放的是各个类别所在图片的文件名，比如cow_val，表示valid数据集中，包含有cow类别目标的图片名称。

<b>3.Annotations</b>

Annotation文件夹中存放着每张图片相关的标注信息，以xml格式的文件存储，可以通过记事本或者浏览器打开，我们以000001.jpg这张图片为例说明标注文件中各个属性的含义，

![avarter](./pic/3-10.png)

1. filename：图片名称

2. size：图片宽高，

3. depth表示图片通道数

4. object：表示目标，包含下面两部分内容。

* 首先是目标类别name为dog。pose表示目标姿势为left，truncated表示是否是一个被截断的目标，1表示是，0表示不是，在这个例子中，只露出狗头部分，所以truncated为1。difficult为0表示此目标不是一个难以识别的目标。

* 然后就是目标的bbox信息，可以看到，这里是以[xmin,ymin,xmax,ymax]格式进行标注的，分别表示dog目标的左上角和右下角坐标。

5. 一张图片中有多少需要识别的目标，其xml文件中就有多少个object。上面的例子中有两个object，分别对应人和狗。

## VOC数据集的dataloader的构建

### 数据集准备

根据上面的介绍可以看出，VOC数据集的存储格式还是比较复杂的，为了后面训练中的读取代码更加简洁，这里我们准备了一个预处理脚本create_data_lists.py。

该脚本的作用是进行一系列的数据准备工作，主要是提前将记录标注信息的xml文件(Annotations)进行解析，并将信息整理到json文件之中，这样在运行训练脚本时，只需简单的从json文件中读取已经按想要的格式存储好的标签信息即可。

注: 这样的预处理并不是必须的，和算法或数据集本身均无关系，只是取决于开发者的代码习惯，不同检测框架的处理方法也是不一致的。

可以看到，create_data_lists.py脚本仅有几行代码，其内部调用了utils.py中的create_data_lists方法：

In [1]:
"""python
    create_data_lists
"""
from utils import create_data_lists

if __name__ == '__main__':
    # voc07_path，voc12_path为我们训练测试所需要用到的数据集，output_folder为我们生成构建dataloader所需文件的路径
    # 参数中涉及的路径以个人实际路径为准，建议将数据集放到dataset目录下，和教程保持一致
    create_data_lists(voc07_path='./dataset/VOCdevkit/VOC2007',
                      voc12_path='./dataset/VOCdevkit/VOC2012',
                      output_folder='./dataset/VOCdevkit')


There are 16551 training images containing a total of 49653 objects. Files have been saved to /home/ma-user/work/dataset/VOCdevkit.

There are 4952 test images containing a total of 14856 objects. Files have been saved to /home/ma-user/work/dataset/VOCdevkit.


### 构建dataloader

In [ ]:
#定义dataloader
#train_dataset和train_loader的实例化
train_dataset = PascalVOCDataset(data_folder,
                                 split='train',
                                 keep_difficult=keep_difficult)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=train_dataset.collate_fn, num_workers=workers,
                                           pin_memory=True)  # note that we're passing the collate function here


实例化PascalVOCDataset类得到train_dataset，然后将train_dataset传入torch.utils.data.DataLoader，进而得到train_loader。

In [ ]:
#PascalVOCDataset
"""python
    PascalVOCDataset具体实现过程
"""
import torch
from torch.utils.data import Dataset
import json
import os
from PIL import Image
from utils import transform


class PascalVOCDataset(Dataset):
    """
    A PyTorch Dataset class to be used in a PyTorch DataLoader to create batches.
    """

    #初始化相关变量
    #读取images和objects标注信息
    def __init__(self, data_folder, split, keep_difficult=False):
        """
        :param data_folder: folder where data files are stored
        :param split: split, one of 'TRAIN' or 'TEST'
        :param keep_difficult: keep or discard objects that are considered difficult to detect?
        """
        self.split = split.upper()    #保证输入为纯大写字母，便于匹配{'TRAIN', 'TEST'}

        assert self.split in {'TRAIN', 'TEST'}

        self.data_folder = data_folder
        self.keep_difficult = keep_difficult

        # Read data files
        with open(os.path.join(data_folder, self.split + '_images.json'), 'r') as j:
            self.images = json.load(j)
        with open(os.path.join(data_folder, self.split + '_objects.json'), 'r') as j:
            self.objects = json.load(j)

        assert len(self.images) == len(self.objects)

    #循环读取image及对应objects
    #对读取的image及objects进行tranform操作（数据增广）
    #返回PIL格式图像，标注框，标注框对应的类别索引，对应的difficult标志(True or False)
    def __getitem__(self, i):
        # Read image
        #*需要注意，在pytorch中，图像的读取要使用Image.open()读取成PIL格式，不能使用opencv
        #*由于Image.open()读取的图片是四通道的(RGBA)，因此需要.convert('RGB')转换为RGB通道
        image = Image.open(self.images[i], mode='r')
        image = image.convert('RGB')

        # Read objects in this image (bounding boxes, labels, difficulties)
        objects = self.objects[i]
        boxes = torch.FloatTensor(objects['boxes'])  # (n_objects, 4)
        labels = torch.LongTensor(objects['labels'])  # (n_objects)
        difficulties = torch.ByteTensor(objects['difficulties'])  # (n_objects)

        # Discard difficult objects, if desired
        #如果self.keep_difficult为False,即不保留difficult标志为True的目标
        #那么这里将对应的目标删去
        if not self.keep_difficult:
            boxes = boxes[1 - difficulties]
            labels = labels[1 - difficulties]
            difficulties = difficulties[1 - difficulties]

        # Apply transformations
        #对读取的图片应用transform
        image, boxes, labels, difficulties = transform(image, boxes, labels, difficulties, split=self.split)

        return image, boxes, labels, difficulties

    #获取图片的总数，用于计算batch数
    def __len__(self):
        return len(self.images)

    #我们知道，我们输入到网络中训练的数据通常是一个batch一起输入，而通过__getitem__我们只读取了一张图片及其objects信息
    #如何将读取的一张张图片及其object信息整合成batch的形式呢？
    #collate_fn就是做这个事情，
    #对于一个batch的images，collate_fn通过torch.stack()将其整合成4维tensor，对应的objects信息分别用一个list存储
    def collate_fn(self, batch):
        """
        Since each image may have a different number of objects, we need a collate function (to be passed to the DataLoader).
        This describes how to combine these tensors of different sizes. We use lists.
        Note: this need not be defined in this Class, can be standalone.
        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of images, lists of varying-size tensors of bounding boxes, labels, and difficulties
        """

        images = list()
        boxes = list()
        labels = list()
        difficulties = list()

        for b in batch:
            images.append(b[0])
            boxes.append(b[1])
            labels.append(b[2])
            difficulties.append(b[3])

        #(3,224,224) -> (N,3,224,224)
        images = torch.stack(images, dim=0)

        return images, boxes, labels, difficulties  # tensor (N, 3, 224, 224), 3 lists of N tensors each

PascalVOCDataset继承了torch.utils.data.Dataset，然后重写了__init__ , __getitem__, __len__ 和 collate_fn 四个方法

In [ ]:
#数据增强
image, boxes, labels, difficulties = transform(image, boxes, labels, difficulties, split=self.split)

transform 函数的具体代码实现位于 utils.py 中，简单讲解

In [ ]:
"""python
    transform操作是训练模型中一项非常重要的工作，其中不仅包含数据增强以提升模型性能的相关操作，也包含如数据类型转换(PIL to Tensor)、归一化(Normalize)这些必要操作。
"""
import json
import os
import torch
import random
import xml.etree.ElementTree as ET
import torchvision.transforms.functional as FT

"""
可以看到，transform分为TRAIN和TEST两种模式，以本实验为例：

在TRAIN时进行的transform有：
1.以随机顺序改变图片亮度，对比度，饱和度和色相，每种都有50％的概率被执行。photometric_distort
2.扩大目标，expand
3.随机裁剪图片，random_crop
4.0.5的概率进行图片翻转，flip
*注意：a. 第一种transform属于像素级别的图像增强，目标相对于图片的位置没有改变，因此bbox坐标不需要变化。
         但是2，3，4，5都属于图片的几何变化，目标相对于图片的位置被改变，因此bbox坐标要进行相应变化。

在TRAIN和TEST时都要进行的transform有：
1.统一图像大小到(224,224)，resize
2.PIL to Tensor
3.归一化，FT.normalize()

注1: resize也是一种几何变化，要知道应用数据增强策略时，哪些属于几何变化，哪些属于像素变化
注2: PIL to Tensor操作，normalize操作必须执行
"""

def transform(image, boxes, labels, difficulties, split):
    """
    Apply the transformations above.
    :param image: image, a PIL Image
    :param boxes: bounding boxes in boundary coordinates, a tensor of dimensions (n_objects, 4)
    :param labels: labels of objects, a tensor of dimensions (n_objects)
    :param difficulties: difficulties of detection of these objects, a tensor of dimensions (n_objects)
    :param split: one of 'TRAIN' or 'TEST', since different sets of transformations are applied
    :return: transformed image, transformed bounding box coordinates, transformed labels, transformed difficulties
    """

    #在训练和测试时使用的transform策略往往不完全相同，所以需要split变量指明是TRAIN还是TEST时的transform方法
    assert split in {'TRAIN', 'TEST'}

    # Mean and standard deviation of ImageNet data that our base VGG from torchvision was trained on
    # see: https://pytorch.org/docs/stable/torchvision/models.html
    #为了防止由于图片之间像素差异过大而导致的训练不稳定问题，图片在送入网络训练之间需要进行归一化
    #对所有图片各通道求mean和std来获得
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    new_image = image
    new_boxes = boxes
    new_labels = labels
    new_difficulties = difficulties

    # Skip the following operations for evaluation/testing
    if split == 'TRAIN':
        # A series of photometric distortions in random order, each with 50% chance of occurrence, as in Caffe repo
        new_image = photometric_distort(new_image)

        # Convert PIL image to Torch tensor
        new_image = FT.to_tensor(new_image)

        # Expand image (zoom out) with a 50% chance - helpful for training detection of small objects
        # Fill surrounding space with the mean of ImageNet data that our base VGG was trained on
        if random.random() < 0.5:
            new_image, new_boxes = expand(new_image, boxes, filler=mean)

        # Randomly crop image (zoom in)
        new_image, new_boxes, new_labels, new_difficulties = random_crop(new_image, new_boxes, new_labels,
                                                                         new_difficulties)

        # Convert Torch tensor to PIL image
        new_image = FT.to_pil_image(new_image)

        # Flip image with a 50% chance
        if random.random() < 0.5:
            new_image, new_boxes = flip(new_image, new_boxes)

    # Resize image to (224, 224) - this also converts absolute boundary coordinates to their fractional form
    new_image, new_boxes = resize(new_image, new_boxes, dims=(224, 224))

    # Convert PIL image to Torch tensor
    new_image = FT.to_tensor(new_image)

    # Normalize by mean and standard deviation of ImageNet data that our base VGG was trained on
    new_image = FT.normalize(new_image, mean=mean, std=std)

    return new_image, new_boxes, new_labels, new_difficulties

In [ ]:
#构建DataLoader
"""python
    DataLoader
"""
#参数说明：
#在train时一般设置shufle=True打乱数据顺序，增强模型的鲁棒性
#num_worker表示读取数据时的线程数，一般根据自己设备配置确定（如果是windows系统，建议设默认值0，防止出错）
#pin_memory，在计算机内存充足的时候设置为True可以加快内存中的tensor转换到GPU的速度，具体原因可以百度哈~
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=train_dataset.collate_fn, num_workers=workers,
                                           pin_memory=True)  # note that we're passing the collate function here